In [1]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

from Backtest.backtest import Backtest
from Backtest.data import OHLCDataHandler
from CCIStrategy import CCIStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import hyperopt.pyll.stochastic

In [3]:
def run_backtest(config, trading_data, ohlc_data, window):
    window = int(window)
    config['title'] = "CCIStrategy" + "_" + str(window)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")
    
    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )
    strategy = CCIStrategy(config, events_queue, data_handler,window = window)

    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler)

    results = backtest.start_trading()
    
    # dict_ans = {
    #     "short_window": [short_window],
    #     "long_window": [long_window],
    #     "Sharpe Ratio": [results['sharpe']],
    #     "Total Returns": [(results['cum_returns'][-1] - 1)],
    #     "Max Drawdown": [(results["max_drawdown"] * 100.0)],
    #     "Max Drawdown Duration": [(results['max_drawdown_duration'])],
    #     "Trades": [results['trade_info']['trading_num']],
    #     "Trade Winning": [results['trade_info']['win_pct']],
    #     "Average Trade": [results['trade_info']['avg_trd_pct']],
    #     "Average Win": [results['trade_info']['avg_win_pct']],
    #     "Average Loss": [results['trade_info']['avg_loss_pct']],
    #     "Best Trade": [results['trade_info']['max_win_pct']],
    #     "Worst Trade": [results['trade_info']['max_loss_pct']],
    #     "Worst Trade Date": [results['trade_info']['max_loss_dt']],
    #     "Avg Days in Trade": [results['trade_info']['avg_dit']]
    # }
    # return pd.DataFrame(dict_ans)
    return -1 * (results['cum_returns'][-1] - 1)

In [9]:
def f(params):
    config = {
        "csv_dir": "./Binance",
        "out_dir": "./results/CCIStrategy",
        "title": "CCIStrategy",
        "is_plot": False,
        "save_plot": False,
        "save_tradelog": False,
        "start_date": pd.Timestamp("2018-02-01T00:0:00", freq = "60" + "T"),    # str(freq) + "T"
        "end_date": pd.Timestamp("2018-06-01T00:00:00", freq = "60" + "T"),
        "equity": 1.0,
        "freq": 30,      # min
        "commission_ratio": 0.001,
        "suggested_quantity": None,     # None or a value
        "max_quantity": None,           # None or a value, Maximum purchase quantity
        "min_quantity": None,           # None or a value, Minimum purchase quantity
        "min_handheld_cash": None,      # None or a value, Minimum handheld funds
        "exchange": "Binance",
        "tickers": ['BTCUSDT']
    }
    # trading_data = {}
    # for ticker in config['tickers']:
    #     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
    #     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

    ohlc_data = {}
    for ticker in config['tickers']:
        # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
        ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '_OHLC_30min.h5', key=ticker)

    trading_data = None
    
    window = params['window']
    val = run_backtest(config, trading_data, ohlc_data, window)
    return {'loss': val, 'status': STATUS_OK}

In [10]:
space = {
    'window': hp.randint('window', 240),
}

In [11]:
# print(hyperopt.pyll.stochastic.sample(space))

In [12]:
trials = Trials()

In [13]:
best = fmin(fn=f, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

---------------------------------
CCIStrategy_86
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-02-01 00:00:00
	End Date	: 2018-06-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.2207456832
Sortino Ratio: -0.3707472752
Information Ratio: 0.0792529594
Max Drawdown: 0.3642720030
Max Drawdown Duration: 494000
Total Returns: -0.1283874003
Annualized Returns: -0.3905116759
Compound Annual Growth Rate: -0.3416088599
---------------------------------
Trades: 29
Trade Winning: 31.03%
Average Trade: -0.12%
Average Win: 7.35%
Average Loss: -3.48%
Best Trade: 21.66%
Worst Trade: -9.37%
Worst Trade Date: 19.0
Avg Days in Trade: 1.7583333333333333
---------------------------------
---------------------------------
CCIStrategy_153
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	:

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.2800472345
Sortino Ratio: 1.7174895252
Information Ratio: 1.4368067581
Max Drawdown: 0.2212380885
Max Drawdown Duration: 223000
Total Returns: 0.1867135164
Annualized Returns: 0.5679202792
Compound Annual Growth Rate: 0.6831987973
---------------------------------
Trades: 16
Trade Winning: 31.25%
Average Trade: 1.50%
Average Win: 11.06%
Average Loss: -2.84%
Best Trade: 20.72%
Worst Trade: -7.68%
Worst Trade Date: 32.0
Avg Days in Trade: 15.437222222222223
---------------------------------
---------------------------------
CCIStrategy_118
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-02-01 00:00:00
	End Date	: 2018-06-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.8807194700
Sortino Ratio: 1.34

D:\Dev\test\backtest\CCIStrategy.py:56: RuntimeWarning: invalid value encountered in double_scalars
  cci = (tp_current - tp_ma) / (0.015 * md)


---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.7567556862
Sortino Ratio: -1.3209783130
Information Ratio: -0.4637200786
Max Drawdown: 0.3787037894
Max Drawdown Duration: 494000
Total Returns: -0.2192583481
Annualized Returns: -0.6669108087
Compound Annual Growth Rate: -0.5289758362
---------------------------------
Trades: 68
Trade Winning: 38.24%
Average Trade: -0.09%
Average Win: 3.75%
Average Loss: -2.47%
Best Trade: 13.00%
Worst Trade: -7.39%
Worst Trade Date: 23.0
Avg Days in Trade: 43.235
---------------------------------
---------------------------------
CCIStrategy_78
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-02-01 00:00:00
	End Date	: 2018-06-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.3737863746
Sortino Ratio: -0.5424244

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.2059786080
Sortino Ratio: -0.2955733298
Information Ratio: 0.0928471091
Max Drawdown: 0.3196986295
Max Drawdown Duration: 494000
Total Returns: -0.1245943353
Annualized Returns: -0.3789744365
Compound Annual Growth Rate: -0.3328551864
---------------------------------
Trades: 33
Trade Winning: 39.39%
Average Trade: -0.05%
Average Win: 4.64%
Average Loss: -3.10%
Best Trade: 14.59%
Worst Trade: -7.36%
Worst Trade Date: 1.0
Avg Days in Trade: 38.78777777777778
---------------------------------
---------------------------------
CCIStrategy_80
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-02-01 00:00:00
	End Date	: 2018-06-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.4842554775
Sortino Ratio: -

...

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.1448259348
Sortino Ratio: 0.1989744169
Information Ratio: 0.3887962914
Max Drawdown: 0.2841207106
Max Drawdown Duration: 241300
Total Returns: -0.0629556491
Annualized Returns: -0.1914900992
Compound Annual Growth Rate: -0.1794523993
---------------------------------
Trades: 15
Trade Winning: 26.67%
Average Trade: 0.05%
Average Win: 10.87%
Average Loss: -3.89%
Best Trade: 19.75%
Worst Trade: -10.58%
Worst Trade Date: 1.0
Avg Days in Trade: 22.466666666666665
---------------------------------
---------------------------------
CCIStrategy_37
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-02-01 00:00:00
	End Date	: 2018-06-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.4021807118
Sortino Ratio: 0.

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.7006703796
Sortino Ratio: 1.0430983762
Information Ratio: 0.9972100042
Max Drawdown: 0.2538694310
Max Drawdown Duration: 239500
Total Returns: 0.0502194288
Annualized Returns: 0.1527507625
Compound Annual Growth Rate: 0.1607182522
---------------------------------
Trades: 20
Trade Winning: 30.00%
Average Trade: 0.63%
Average Win: 9.67%
Average Loss: -3.25%
Best Trade: 20.72%
Worst Trade: -10.02%
Worst Trade Date: 32.0
Avg Days in Trade: 38.75
---------------------------------
---------------------------------
CCIStrategy_155
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-02-01 00:00:00
	End Date	: 2018-06-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.5904273761
Sortino Ratio: 0.7939506739
Info

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.8696126387
Sortino Ratio: 1.3348848161
Information Ratio: 1.2014094559
Max Drawdown: 0.2463791348
Max Drawdown Duration: 238400
Total Returns: 0.0899927597
Annualized Returns: 0.2737279775
Compound Annual Growth Rate: 0.2996612088
---------------------------------
Trades: 20
Trade Winning: 30.00%
Average Trade: 0.83%
Average Win: 10.20%
Average Loss: -3.19%
Best Trade: 22.62%
Worst Trade: -9.83%
Worst Trade Date: 34.0
Avg Days in Trade: 41.7
---------------------------------
---------------------------------
CCIStrategy_52
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-02-01 00:00:00
	End Date	: 2018-06-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.1788050436
Sortino Ratio: 4.4442057225
Inform

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.6731174013
Sortino Ratio: 0.8654340632
Information Ratio: 0.8133782571
Max Drawdown: 0.2146657099
Max Drawdown Duration: 418500
Total Returns: 0.0422308392
Annualized Returns: 0.1284521359
Compound Annual Growth Rate: 0.1340710275
---------------------------------
Trades: 16
Trade Winning: 31.25%
Average Trade: 0.64%
Average Win: 8.63%
Average Loss: -3.00%
Best Trade: 21.44%
Worst Trade: -7.68%
Worst Trade Date: 32.0
Avg Days in Trade: 13.625
---------------------------------
---------------------------------
CCIStrategy_180
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-02-01 00:00:00
	End Date	: 2018-06-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.1343389494
Sortino Ratio: 0.1792075458
Info

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.3415183484
Sortino Ratio: 0.4442740248
Information Ratio: 0.5213489450
Max Drawdown: 0.2290191394
Max Drawdown Duration: 418500
Total Returns: -0.0213967016
Annualized Returns: -0.0650816341
Compound Annual Growth Rate: -0.0636706496
---------------------------------
Trades: 16
Trade Winning: 31.25%
Average Trade: 0.28%
Average Win: 8.18%
Average Loss: -3.31%
Best Trade: 19.48%
Worst Trade: -9.06%
Worst Trade Date: 33.0
Avg Days in Trade: 7.5
---------------------------------
---------------------------------
CCIStrategy_40
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-02-01 00:00:00
	End Date	: 2018-06-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.5314587450
Sortino Ratio: 1.1379466721
Infor

In [ ]:
best

In [ ]:
for trial in trials.trials:
    print(trial['misc']['vals'], "\t",trial['result']['loss'])

In [ ]:
config = {
    "csv_dir": "./Binance",
    "out_dir": "./results/CCIStrategy",
    "title": "CCIStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2018-04-01T00:0:00", freq = "60" + "T"),    # str(freq) + "T"
    "end_date": pd.Timestamp("2018-08-01T00:00:00", freq = "60" + "T"),
    "equity": 1.0,
    "freq": 60,      # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}
ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None
run_backtest(config,ohlc_data,ohlc_data,53)

In [ ]:
f, ax = plt.subplots(1,2, figsize = (16, 8))
xs = [t['misc']['vals']['window'] for t in trials.trials]
ys = [-1 * t['result']['loss'] for t in trials.trials]
ax[0].scatter(xs, ys, s=20, linewidth=0.01, alpha=0.75)
ax[0].set_title('$val$ $vs$ $window$ ', fontsize=18)
ax[0].set_xlabel('$window$', fontsize=16)
ax[0].set_ylabel('$val$', fontsize=16)
ax[0].hlines(0, 0, 120, color='black',linestyle="--")
ax[0].hlines(-1, 0, 120, color='black',linestyle="--")
ax[0].hlines(1, 0, 120, color='black',linestyle="--")
ax[0].vlines(best['window'], -1.2, 2, color='blue',linestyle="--")